In [7]:
def seed_everything(seed = 0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

In [8]:
from torch.utils.data import Dataset, DataLoader
from glob import glob
from albumentations import Compose, Resize, HorizontalFlip, Rotate
from tqdm.notebook import tqdm
import random
import numpy as np
import torch
import pickle
import timm

In [9]:
class param:
    num_epoch = 8
    batch_size = 64
    num_data = 12800
    img_size = 384 #224
    img_face_crop_mergin = 60
    img_center_crop_size = 224
    num_workers = 4
    lr = 0.0001
    k_fold = 5
    train_patience = 4

In [13]:
seed_everything()

class path:
    train_csv = '/opt/ml/input/data/train/train.csv'
    train_images = '/opt/ml/input/data/train/images'
    eval_images = '/opt/ml/input/data/eval/images'
    model_save = '/opt/ml/code/results'

class FaceDataset(Dataset):
    def __init__(self, imgs):
        self.imgs = imgs
        self.bboxes = [pickle.load(open(os.path.splitext(img)[0] + '.txt', 'rb')) for img in self.imgs]

    def __getitem__(self, idx):
        img = Image.open(self.imgs[idx])
        img_size = img.size
        box = self.bboxes[idx]
        if box is not None:
            margin = param.img_face_crop_mergin
            box = [
                int(max(box[0] - margin / 2, 0)),
                int(max(box[1] - margin / 2, 0)),
                int(min(box[2] + margin / 2, img_size[0])),
                int(min(box[3] + margin / 2, img_size[1]))
            ]
        else:
            margin = param.img_center_crop_size
            box = [
                int(img_size[0] / 2 - margin / 2),
                int(img_size[1] / 2 - margin / 2),
                int(img_size[0] / 2 + margin / 2),
                int(img_size[1] / 2 + margin / 2)
            ]
        img = img.crop(box)
        return img
    
    def __len__(self):
        return len(self.imgs)

def get_eval_dataset(images_path):
    img_list = glob(images_path + '/*.jpg')
    return FaceDataset(img_list)

transforms_list = [
]
    '''Rotate([5, 5], p=1.0),
    Rotate([-5, -5], p=1.0),
    HorizontalFlip(p=1.0),
    Compose([Rotate([5, 5], p=1.0), HorizontalFlip(p=1.0)]),
    Compose([Rotate([-5, -5], p=1.0), HorizontalFlip(p=1.0)])'''

eval_dataset = get_eval_dataset(path.eval_images)

IndentationError: unexpected indent (<ipython-input-13-52b053f2d522>, line 46)

In [15]:
eval_dataloader = DataLoader(eval_dataset, shuffle=True, num_workers=param.num_workers)

model_paths = [
    '/opt/ml/code/results/last_0_224/005_accuracy_65.43%.ckpt'
]

'''
    ,
    '/opt/ml/code/results/last_0_384/007_accuracy_66.80%.ckpt',
    '/opt/ml/code/results/last_1_224/007_accuracy_67.19%.ckpt',
    '/opt/ml/code/results/last_1_384/006_accuracy_66.60%.ckpt',
    '/opt/ml/code/results/last_2_224/006_accuracy_69.53%.ckpt',
    '/opt/ml/code/results/last_2_384/006_accuracy_70.70%.ckpt',
    '/opt/ml/code/results/last_3_224/007_accuracy_68.55%.ckpt',
    '/opt/ml/code/results/last_3_384/004_accuracy_65.62%.ckpt',
    '/opt/ml/code/results/last_4_224/006_accuracy_67.97%.ckpt',
    '/opt/ml/code/results/last_4_384/005_accuracy_68.95%.ckpt',
    '/opt/ml/code/results/last_0_512/007_accuracy_66.41%.ckpt',
    '/opt/ml/code/results/last_1_512/006_accuracy_63.87%.ckpt',
    '/opt/ml/code/results/last_2_512/007_accuracy_70.12%.ckpt'

'''
model_list = []
for model_path in model_paths:
    model = timm.create_model('efficientnet_b3', num_classes=18)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    model_list.append(model)
    

device = torch.device('cuda')
all_predictions = []
for images in tqdm(eval_dataloader):
    images = images.to(device)
    num_predict = 0
    for model in path:
        pred = model(images)
        for transforms in transforms_list:
            pred += model(transforms(images))
            num_predict += 1
    
    pred /= num_predict
    all_predictions.extend(pred.cpu().numpy())

submission['ans'] = np.argmax(all_prediction, axis=1)
submission.to_csv(os.path.join(path.eval_images, 'submission.csv'), index=False)
print('test inference is done!')


RuntimeError: CUDA error: out of memory

In [7]:
from collections import Counter
from glob import glob
img_list = glob(path.eval_images + '/*')
ext_list = []
for img in img_list:
    ext_list.append(os.path.splitext(img)[1])
ext_list = Counter(ext_list)
print(ext_list)

Counter({'.jpg': 12600, '.txt': 12600})


In [4]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:
torch.cuda.empty_cache()